In [ ]:
import pandas as pd

input_csv = r"C:\Users\danie\OneDrive\Desktop\All Countries.csv"
output_csv = r"C:\Users\danie\OneDrive\Desktop\country_data_clean_iso.csv"
iso_csv = r"C:\Users\danie\OneDrive\Desktop\iso_codes.csv"

df = pd.read_csv(input_csv, dtype=str, keep_default_na=False, engine="python")
df.to_csv(output_csv, index=False)

df_iso = pd.read_csv(iso_csv, dtype=str, keep_default_na=False, engine="python")

df_merged = df.merge(df_iso, how="left", left_on="country", right_on="Name")
df_merged = df_merged.drop(columns=["Name"])

df_merged.to_csv(output_csv, index=False)

manual_iso = {
    "Bolivia": "BO",
    "Cabo Verde": "CV",
    "Dem. People's Rep. Korea": "KP",
    "Dem. Rep. Congo": "CD",
    "Iran": "IR",
    "Korea": "KR",
    "Micronesia": "FM",
    "Moldova": "MD",
    "North Macedonia": "MK",
    "Russia": "RU",
    "São Tomé and Principe": "ST",
    "Slovak Republic": "SK",
    "St. Kitts and Nevis": "KN",
    "St. Lucia": "LC",
    "St. Vincent and the Grenadines": "VC",
    "Tanzania": "TZ",
    "The Bahamas": "BS",
    "The Gambia": "GM",
    "Venezuela": "VE",
    "Vietnam": "VN",
    "West Bank and Gaza": "PS",
    "Brunei": "BN",
    "Kyrgyz Republic": "KG",
    "Lao PDR": "LA",
}

# Fill blanks in Code using the manual map
mask_missing = df_merged["Code"].isna() | (df_merged["Code"] == "")
df_merged.loc[mask_missing, "Code"] = df_merged.loc[mask_missing, "country"].map(manual_iso)

df_merged = df_merged.rename(columns={"Code": "iso_code"})

# Save final dataset
df_merged.to_csv(output_csv, index=False)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
from unicodedata import normalize

df = pd.read_csv(r"C:\Users\danie\OneDrive\Desktop\2020-2025.csv", encoding="utf-8-sig")

# Standardize expected column names
df = df.rename(columns={"2024": "gdp_2024", "2025": "gdp_2025", "Country": "country"})
df['country'] = df['country'].astype(str).str.strip()

# Clean GDP columns to numeric
for col in ["gdp_2024", "gdp_2025"]:
    df[col] = (
        df[col].astype(str)
        .str.replace('\u00A0','', regex=False)
        .str.replace('\u2009','', regex=False)
        .str.replace(',', '', regex=False)
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Normalize accents & case once
def norm(s):
    if pd.isna(s): return ""
    return normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii").upper().strip()

aliases_to_iso2 = {
    # Straight mappings / alternate names
    "BRUNEI": "BN", "BRUNEI DARUSSALAM": "BN",
    "RUSSIA": "RU", "RUSSIAN FEDERATION": "RU", "RUSSIA (FEDERATION)": "RU",
    "CAPE VERDE": "CV", "CABO VERDE": "CV",
    "DEMOCRATIC REPUBLIC OF THE CONGO": "CD", "CONGO, DEM. REP.": "CD", "CONGO (KINSHASA)": "CD",
    "IVORY COAST": "CI", "COTE D'IVOIRE": "CI", "COTE DIVOIRE": "CI",
    "KOSOVO": "XK", "KOSOVA": "XK",
    "MACAU": "MO", "MACAO": "MO",
    "PALESTINE": "PS", "STATE OF PALESTINE": "PS", "PALESTINIAN TERRITORIES": "PS", "WEST BANK AND GAZA": "PS",
    "SAO TOME AND PRINCIPE": "ST", "SAO TOME & PRINCIPE": "ST",
    "TURKEY": "TR", "TURKIYE": "TR",
}

def to_iso2(name: str) -> str | None:
    if pd.isna(name): return None
    s = str(name).strip()
    su = norm(s)

    # 1) try pycountry’s own resolver
    try:
        return pycountry.countries.lookup(s).alpha_2
    except Exception:
        pass

    # 2) try alias on normalized form (handles accents & punctuation)
    return aliases_to_iso2.get(su)

# Apply
df['iso2_code'] = df['country'].apply(to_iso2)

# Verify remaining issues (should now be empty)
unmapped = (df[df['iso2_code'].isna()][['country']]
            .drop_duplicates().sort_values('country'))
print(unmapped.to_string(index=False))

# Save clean file for SQL load
out = df[['iso2_code', 'gdp_2024', 'gdp_2025']].dropna(subset=['iso2_code'])
out = out.rename(columns={'iso2_code': 'iso_code'})   # standardise to match SQL

out_path = r"C:\Users\danie\OneDrive\Desktop\gdp_clean_iso.csv"
out.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"Clean file written: {out_path}  | rows: {len(out)}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

CSV_PATH = r"C:\Users\danie\OneDrive\Desktop\plotting.csv"

N = 15
REGION_CANDIDATES = ["region", "Region", "continent", "Continent", "subregion", "Subregion"]

def clean_pct(series: pd.Series) -> pd.Series:
    s = (series.astype(str)
               .str.replace("%", "", regex=False)
               .str.replace("\u00A0", "", regex=False) 
               .str.replace("\u2009", "", regex=False) 
               .str.replace(",", "", regex=False)
               .str.strip()
               .str.replace(r"^\((.*)\)$", r"-\1", regex=True))  
    return pd.to_numeric(s, errors="coerce").clip(lower=0, upper=100)

# Load
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")

# Country column
country_col = next((c for c in ["country_long", "country", "Country", "country_name"] if c in df.columns), None)
if not country_col:
    raise KeyError("No country name column found (expected one of country_long/country/Country/country_name).")

# Required columns
req = ["electricity_access_pct", "internet_access_pct"]
missing = [c for c in req if c not in df.columns]
if missing:
    raise KeyError(f"Missing columns in plotting.csv: {missing}")

# Filter: Africa only
region_col = next((c for c in REGION_CANDIDATES if c in df.columns), None)
if not region_col:
    raise KeyError(f"No region/continent column found (checked {REGION_CANDIDATES}).")

df = df[df[region_col].astype(str).str.contains("africa", case=False, na=False)]
if df.empty:
    raise ValueError("After filtering by region contains 'Africa', no rows remain. Check your region values.")

# Clean metrics
df["electricity_access_pct"] = clean_pct(df["electricity_access_pct"])
df["internet_access_pct"]    = clean_pct(df["internet_access_pct"])
df = df.dropna(subset=["electricity_access_pct", "internet_access_pct"]).copy()

# Compute bottleneck score
df["both_high_score"] = df[["electricity_access_pct", "internet_access_pct"]].min(axis=1)

# Rank
top = df.nlargest(N, "both_high_score").copy().sort_values("both_high_score")

# Plot
countries = top[country_col].tolist()
elec = top["electricity_access_pct"].to_numpy()
net  = top["internet_access_pct"].to_numpy()
y = np.arange(len(countries))
bar_h = 0.35

# Dynamic threshold = weakest "both_high_score" in Top N
threshold = float(top["both_high_score"].min())

plt.figure(figsize=(10, max(6, 0.45 * len(countries))))
plt.barh(y - bar_h/2, elec, height=bar_h, label="Electricity access (%)")
plt.barh(y + bar_h/2, net,  height=bar_h, label="Internet access (%)")
plt.axvline(threshold, linewidth=0.8, linestyle="--", color="grey", label=f"Threshold = {threshold:.1f}%")
plt.yticks(y, countries)
plt.xlabel("Access (%)")
plt.title(f"Africa — Top {len(top)} by Electricity & Internet Access\n(ranked by min of the two)")
plt.legend(loc="lower right")
plt.xlim(0, 100)
plt.tight_layout()

# Annotate values
ax = plt.gca()
for i, (e, n) in enumerate(zip(elec, net)):
    ax.text(e + 1, y[i] - bar_h/2, f"{e:.1f}%", va="center")
    ax.text(n + 1, y[i] + bar_h/2, f"{n:.1f}%", va="center")

plt.show()

# Print table
print(
    top[[country_col, region_col, "electricity_access_pct", "internet_access_pct", "both_high_score"]]
      .rename(columns={country_col: "country"})
      .to_string(index=False, formatters={
          "electricity_access_pct": "{:.1f}".format,
          "internet_access_pct": "{:.1f}".format,
          "both_high_score": "{:.1f}".format
      })
)
print(f"\nApplied access threshold (min of the two metrics among Top {len(top)}): {threshold:.1f}%")